In [14]:
import pandas as pd

In [19]:
df = pd.read_excel('miuul_gezinomi.xlsx')

# used to display the first few rows of data


In [20]:
print(df.head())

   SaleId   SaleDate CheckInDate  Price     ConceptName SaleCityName  \
0  415122 2022-12-03  2022-12-03  79.30    Herşey Dahil      Antalya   
1  415103 2022-12-03  2022-12-03  45.97  Yarım Pansiyon      Antalya   
2  404034 2022-09-12  2022-09-13  77.84    Herşey Dahil      Antalya   
3  415094 2022-12-03  2022-12-10 222.71  Yarım Pansiyon        İzmir   
4  414951 2022-12-01  2022-12-03 140.48  Yarım Pansiyon        İzmir   

     CInDay  SaleCheckInDayDiff Seasons  
0  Saturday                   0     Low  
1  Saturday                   0     Low  
2   Tuesday                   1    High  
3  Saturday                   7     Low  
4  Saturday                   2     Low  


# determine the dimensions (number of rows and columns) of a dataset.
 

In [22]:
print(df.shape)

(59164, 9)


# provide detailed information about a dataset. 

In [23]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB
None


# How many unique cities are there? What are their frequencies?

In [25]:
print(df['SaleCityName'].nunique())


6


# What are their frequencies?


In [26]:
print(df['SaleCityName'].value_counts())

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64


# How many unique Concepts are there?


In [27]:
print(df["ConceptName"].nunique())

3


# How many sales were realized from which Concept?

In [28]:
print(df['ConceptName'].value_counts())

Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: ConceptName, dtype: int64


# How much was earned in total from sales by cities?

In [30]:
df.groupby('SaleCityName').aggregate({'Price' : 'sum' })

,Price
SaleCityName,
Antalya,2041911.10
Aydın,573296.01
Diğer,154572.29
Girne,27065.03
Muğla,665842.21
İzmir,165934.83


# How much is earned according to concept types?

In [31]:
df.groupby('ConceptName').aggregate({'Price' : 'sum' })

,Price
ConceptName,
Herşey Dahil,3332910.77
Oda + Kahvaltı,121308.35
Yarım Pansiyon,174402.35


# What are the PRICE averages by city?

In [32]:
df.groupby('SaleCityName').aggregate({'Price' : 'mean' })

,Price
SaleCityName,
Antalya,64.52
Aydın,53.86
Diğer,47.71
Girne,59.48
Muğla,62.46
İzmir,66.27


# What are the PRICE averages by concepts? 

In [33]:
df.groupby('ConceptName').aggregate({'Price' : 'mean' })

,Price
ConceptName,
Herşey Dahil,62.67
Oda + Kahvaltı,50.25
Yarım Pansiyon,49.03


# What are the PRICE averages in the City-Concept breakdown?

In [35]:
df.groupby(by=['SaleCityName','ConceptName']).aggregate({'Price' : 'mean' })

Price
SaleCityName ConceptName          
Antalya      Herşey Dahil    64.52
             Oda + Kahvaltı  63.50
             Yarım Pansiyon  67.19
Aydın        Herşey Dahil    54.00
             Oda + Kahvaltı  34.46
             Yarım Pansiyon  30.02
Diğer        Herşey Dahil    84.77
             Oda + Kahvaltı  37.60
             Yarım Pansiyon  42.11
Girne        Herşey Dahil    97.68
             Oda + Kahvaltı  39.78
             Yarım Pansiyon  53.25
Muğla        Herşey Dahil    63.02
             Oda + Kahvaltı  59.04
             Yarım Pansiyon  45.12
İzmir        Herşey Dahil    74.70
             Oda + Kahvaltı  41.32
             Yarım Pansiyon  59.61

# Convert the SaleCheckInDayDiff variable to a categorical variable

In [36]:
bins = [-1, 7, 30, 90, df['SaleCheckInDayDiff'].max()]
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]


In [38]:
df["EB_Score"] = pd.cut(df["SaleCheckInDayDiff"], bins, labels = labels)
df.head(50).to_excel("eb_score.xlsx", index = False)

# In terms of average fees paid and number of transactions in City-Concept-EB Point breakdown review

In [39]:
df.groupby(by=['SaleCityName','ConceptName',"EB_Score"]).aggregate({'Price' :['mean', 'count']})

Price       
                                                mean  count
SaleCityName ConceptName    EB_Score                       
Antalya      Herşey Dahil   Last Minuters      62.75  14148
                            Potential Planners 64.90   8874
                            Planners           67.88   4490
                            Early Bookers      66.49   3281
             Oda + Kahvaltı Last Minuters      65.35    503
...                                              ...    ...
İzmir        Oda + Kahvaltı Early Bookers      66.93      3
             Yarım Pansiyon Last Minuters      57.79    636
                            Potential Planners 64.80    198
                            Planners           60.51     47
                            Early Bookers      65.91     14

[72 rows x 2 columns]

# Rank the output of the City-Concept-Season breakdown by PRICE.

In [42]:
agg_df = df.groupby(by=['SaleCityName','ConceptName',"Seasons"]).aggregate({'Price' : 'mean'}).sort_values('Price')
agg_df

Price
SaleCityName ConceptName    Seasons       
Aydın        Yarım Pansiyon Low      25.27
             Oda + Kahvaltı High     30.39
             Yarım Pansiyon High     32.63
Muğla        Yarım Pansiyon Low      32.68
İzmir        Oda + Kahvaltı Low      33.56
Aydın        Herşey Dahil   Low      33.68
Diğer        Oda + Kahvaltı High     34.84
Muğla        Oda + Kahvaltı Low      37.77
Girne        Oda + Kahvaltı High     39.48
Diğer        Yarım Pansiyon High     39.73
             Oda + Kahvaltı Low      39.96
Girne        Oda + Kahvaltı Low      40.04
Diğer        Yarım Pansiyon Low      42.95
Aydın        Oda + Kahvaltı Low      44.45
Muğla        Yarım Pansiyon High     45.82
             Herşey Dahil   Low      48.30
Girne        Yarım Pansiyon Low      48.58
İzmir        Oda + Kahvaltı High     54.14
Aydın        Herşey Dahil   High     54.95
İzmir        Yarım Pansiyon Low      55.35
Girne        Yarım Pansiyon High     58.38
Antalya      Oda + Kahvaltı Low      60.67
Muğla        Oda + Kahvaltı High     61.26
Antalya      Herşey Dahil   Low      61.55
             Yarım Pansiyon Low      62.00
Muğla        Herşey Dahil   High     63.38
Antalya      Herşey Dahil   High     64.92
             Oda + Kahvaltı High     66.27
             Yarım Pansiyon High     73.26
İzmir        Herşey Dahil   Low      74.31
                            High     74.75
Diğer        Herşey Dahil   High     83.79
                            Low      87.31
İzmir        Yarım Pansiyon High     87.66
Girne        Herşey Dahil   Low      90.94
                            High    103.94

# Convert the names in the index to variable names.

In [44]:
agg_df.reset_index(inplace = True)
agg_df

,index,SaleCityName,ConceptName,Seasons,Price
0,0,Aydın,Yarım Pansiyon,Low,25.27
1,1,Aydın,Oda + Kahvaltı,High,30.39
2,2,Aydın,Yarım Pansiyon,High,32.63
3,3,Muğla,Yarım Pansiyon,Low,32.68
4,4,İzmir,Oda + Kahvaltı,Low,33.56
5,5,Aydın,Herşey Dahil,Low,33.68
6,6,Diğer,Oda + Kahvaltı,High,34.84
7,7,Muğla,Oda + Kahvaltı,Low,37.77
8,8,Girne,Oda + Kahvaltı,High,39.48
9,9,Diğer,Yarım Pansiyon,High,39.73


# Identify new level-based customers (personas)

In [48]:
agg_df["sales_level_based"] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].aggregate(lambda x : '_'.join(x).upper(), axis = 1)

agg_df.head(15)

,index,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,0,Aydın,Yarım Pansiyon,Low,25.27,AYDIN_YARIM PANSIYON_LOW
1,1,Aydın,Oda + Kahvaltı,High,30.39,AYDIN_ODA + KAHVALTI_HIGH
2,2,Aydın,Yarım Pansiyon,High,32.63,AYDIN_YARIM PANSIYON_HIGH
3,3,Muğla,Yarım Pansiyon,Low,32.68,MUĞLA_YARIM PANSIYON_LOW
4,4,İzmir,Oda + Kahvaltı,Low,33.56,İZMIR_ODA + KAHVALTI_LOW
5,5,Aydın,Herşey Dahil,Low,33.68,AYDIN_HERŞEY DAHIL_LOW
6,6,Diğer,Oda + Kahvaltı,High,34.84,DIĞER_ODA + KAHVALTI_HIGH
7,7,Muğla,Oda + Kahvaltı,Low,37.77,MUĞLA_ODA + KAHVALTI_LOW
8,8,Girne,Oda + Kahvaltı,High,39.48,GIRNE_ODA + KAHVALTI_HIGH
9,9,Diğer,Yarım Pansiyon,High,39.73,DIĞER_YARIM PANSIYON_HIGH


# Segment new customers 

In [50]:
agg_df['SEGMENT'] = pd.qcut(agg_df['Price'], 4, labels = ["D","C","B","A"])
agg_df

,index,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
0,0,Aydın,Yarım Pansiyon,Low,25.27,AYDIN_YARIM PANSIYON_LOW,D
1,1,Aydın,Oda + Kahvaltı,High,30.39,AYDIN_ODA + KAHVALTI_HIGH,D
2,2,Aydın,Yarım Pansiyon,High,32.63,AYDIN_YARIM PANSIYON_HIGH,D
3,3,Muğla,Yarım Pansiyon,Low,32.68,MUĞLA_YARIM PANSIYON_LOW,D
4,4,İzmir,Oda + Kahvaltı,Low,33.56,İZMIR_ODA + KAHVALTI_LOW,D
5,5,Aydın,Herşey Dahil,Low,33.68,AYDIN_HERŞEY DAHIL_LOW,D
6,6,Diğer,Oda + Kahvaltı,High,34.84,DIĞER_ODA + KAHVALTI_HIGH,D
7,7,Muğla,Oda + Kahvaltı,Low,37.77,MUĞLA_ODA + KAHVALTI_LOW,D
8,8,Girne,Oda + Kahvaltı,High,39.48,GIRNE_ODA + KAHVALTI_HIGH,D
9,9,Diğer,Yarım Pansiyon,High,39.73,DIĞER_YARIM PANSIYON_HIGH,C


In [52]:
agg_df.groupby('SEGMENT').aggregate({'Price' : ['mean', 'max', 'sum']})

Price              
         mean    max    sum
SEGMENT                    
D       33.37  39.48 300.30
C       44.89  54.14 403.99
B       60.27  64.92 542.47
A       82.47 103.94 742.21